# **Save this file as studentid1_studentid2_lab#.ipynb**
(Your student-id is the number shown on your student card.)

E.g. if you work with 2 people, the notebook should be named:
12301230_3434343_lab1.ipynb.

**IMPORTANT: This will be parsed by a regexp, so please double check your filename.**

**Only one member of each group has to submit the file to canvas.**

Before you turn this problem in, please make sure everything runs correctly. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Note, that **you are not allowed to use Google Colab**.

**Make sure you fill in any place that says `YOUR CODE HERE`.**

**Finally, please add your names and email adresses below.**



NAME = Andrei Blahovici
NAME2 = Demi Kruijer
EMAIL = blahoviciandrei1@gmail.com
EMAIL2 = daekruijer@gmail.com

# Lab 1: Linear Regression and Overfitting

### Machine Learning 1, September/October 2022

Notes on implementation:

* You should write your code and answers in this IPython Notebook: http://ipython.org/notebook.html. If you have problems, please contact your TA.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* Refer to last week's lab notes, i.e. http://docs.scipy.org/doc/, if you are unsure about what function to use. There are different correct ways to implement each problem!
* For this lab, your regression solutions should be in closed form, i.e., should not perform iterative gradient-based optimization but find the exact optimum directly.
*Use the provided test boxes to check if your answers are correct.
* **Do not change the arugments in our functions!**
* **Do not remove add new cells. If you do so you should expect a penalty from ourside!**

### Relevant materials for this assignment

* Erik's video lectures Week 3 and 4 
* Christopher Bishop book: Pattern recognition and machine learning (Chapter 3)
* Mathematics for machine learning (Section 8.3 and 8.4)

In [ ]:
%pylab inline
plt.rcParams["figure.figsize"] = [20,10]

In [ ]:
# This cell makes sure that you have all the necessary libraries installed

import sys
import platform
from importlib.util import find_spec, module_from_spec

def check_newer_version(version_inst, version_nec):
    version_inst_split = version_inst.split('.')
    version_nec_split = version_nec.split('.')
    for i in range(min(len(version_inst_split), len(version_nec_split))):
        if int(version_nec_split[i]) > int(version_inst_split[i]):
            return False
        elif int(version_nec_split[i]) < int(version_inst_split[i]):
            return True
    return True


module_list = [('jupyter', '1.0.0'), 
               ('matplotlib', '3.5.1'), 
               ('numpy', '1.22.3'), 
               ('python', '3.9.5'), 
               ('sklearn', '1.1.1'), 
               ('scipy', '1.7.3'), 
               ('nb_conda', '2.2.1')]

packages_correct = True
packages_errors = []

for module_name, version in module_list:
    if module_name == 'scikit-learn':
        module_name = 'sklearn'
    if 'python' in module_name:
        python_version = platform.python_version()
        if not check_newer_version(python_version, version):
            packages_correct = False
            error = f'Update {module_name} to version {version}. Current version is {python_version}.'
            packages_errors.append(error) 
            print(error)
    else:
        spec = find_spec(module_name)
        if spec is None:
            packages_correct = False
            error = f'Install {module_name} with version {version} or newer, it is required for this assignment!'
            packages_errors.append(error) 
            print(error)
        else:
            x = __import__(module_name)
            if hasattr(x, '__version__') and not check_newer_version(x.__version__, version):
                packages_correct = False
                error = f'Update {module_name} to version {version}. Current version is {x.__version__}.'
                packages_errors.append(error) 
                print(error)

try:
    from google.colab import drive
    packages_correct = False
    error = """Please, don't use google colab!
It will make it much more complicated for us to check your homework as it merges all the cells into one."""
    packages_errors.append(error) 
    print(error)
except:
    pass

packages_errors = '\n'.join(packages_errors)

Relevant materials for this part:

* Erik's lecture 3.1 and 3.2
* Section 1.1 and 3.1 from Bishop's book Pattern recognition for machine learning.
* Mathematics for machine learning (Section 8.3)


### 1.1. Generate periodic data (5 points)
Write a method `gen_sine(N)` that generates toy data like in fig 1.2 of Bishop's book. The method should have a parameter $N$, and should return $N$-dimensional vectors $\mathbf{x}$ and $\mathbf{t}$, where $\mathbf{x}$ contains evenly spaced values from 0 to (including) 2$\pi$, and the elements $t_i$ of $\mathbf{t}$ are distributed according to:

$$t_i \sim \mathcal{N}(\mu_i, \sigma^2)$$

where $x_i$ is the $i$-th elements of $\bf{x}$, the mean $\mu_i = \sin(x_i)$ and the standard deviation $\sigma = 0.25$. You can make use of `np.random.normal()` (Hint: Double check its input parameters).


In [ ]:
def gen_sine(n):
  # define x as evenly spaced vector of length n from 0 to 2pi 
  x =  np.linspace(0,2*np.pi, n)
  # define y as sine of x plus random noise
  t = np.sin(x) + np.random.normal(0, 0.25, n)

  return x, t

In [ ]:
### Test your function
np.random.seed(42)
N = 10
x, t = gen_sine(N)

assert x.shape == (N,), "the shape of x is incorrect"
assert t.shape == (N,), "the shape of t is incorrect"



### 1.2 Polynomial regression (10 points)

Write a method `fit_polynomial(x, t, M)` that finds the maximum-likelihood solution of an _unregularized_ $M$-th order polynomial for some dataset `x`. The error function to minimize w.r.t. $\mathbf{w}$ is:

$E(\mathbf{w}) = \frac{1}{2} (\mathbf{\Phi} \mathbf{w} - \mathbf{t})^T(\mathbf{\Phi} \mathbf{w} - \mathbf{t})$

where $\mathbf{\Phi}$ is the _feature matrix_ (or _design matrix_) as explained in Bishop's book at section 3.1.1, $\mathbf{t}$ is the vector of target values. Your method should return a vector $\mathbf{w}$ with the maximum-likelihood parameter estimates, as well as the _feature matrix_ $\mathbf{\Phi}$.

In [ ]:
def designmatrix(x, M): 
  # initialize matrix with bias vector of ones 
  matrix = np.array([np.ones(len(x))])
  # iteratively fill columns where col1 is x^1, col2 is x^2 etc.
  for m in range(1,M+1):
    column = x**m
    #print('column is:',column)
    matrix = np.concatenate((matrix, [column]), axis=0)
  # columns were added as rows, so tranpose the whole thing 
  matrix = matrix.transpose()

  return matrix #, matrix.shape

def fit_polynomial(x, t, M):
  # define matrices 
  Phi = designmatrix(x,M)
  Phi_T = Phi.transpose()
  # calculate pseudo inverse and weights 
  pseudo_inverse = np.matmul(np.linalg.inv(np.matmul(Phi_T, Phi)), Phi_T)
  w_ml = np.matmul(pseudo_inverse, t) 

  return w_ml, Phi

In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
m = 2
w, Phi = fit_polynomial(x,t,m)

assert w.shape == (m+1,), "The shape of w is incorrect"
assert Phi.shape == (N, m+1), "The shape of Phi is incorrect"



### 1.3 Plot (5 points)
Sample a dataset with $N=10$, and fit four polynomials with $M \in (0, 2, 4, 8)$.
For each value of $M$, plot the prediction function, along with the data and the original sine function. The resulting figure should look similar to fig 1.4 of the Bishop's book. Note that you can use matplotlib's `plt.pyplot(.)` functionality for creating grids of figures.

In [ ]:
# define datapoints
data_x, data_t = gen_sine(10)
x_ticks = np.arange(0,2*np.pi,0.1)   
sin_y = np.sin(x_ticks)

# create polynomial of given order 
def polynomial(x,w):
  y = np.ones(len(x))*w[0]
  for factor in range(1,len(w)):
      y = y + w[factor] * x**factor
  
  return y

plotcount = 221
# iteratively plot polynomials
for order in [0,2,4,8]:    
  # generate plots of datapoints and sine
  plt.subplot(plotcount)
  plotcount = plotcount + 1
  # plot data and sine
  plt.plot(data_x, data_t, 'ro')
  plt.plot(x_ticks, sin_y)
  # calculate weights and plot polynomial
  w_ml = fit_polynomial(data_x,data_t,order)[0]
  plt.plot(x_ticks,polynomial(x_ticks,w_ml))
  # set labels 
  plt.xlabel('x')
  plt.ylabel('t')
  plt.text(4, 0.5, 'M=%s'%(order), fontsize = 10)

plt.show()

### 1.4 Regularized linear regression (15 points)

Relevant material for this part:

* Lecture 3.5 (Regularized Least Squares)
* Section 1.1 and 3.1.4 from Bishop's book Pattern recognition for machine learning.
* Mathematics for machine learning (Section 8.3.2)

a) (10 points) Write a method `fit_polynomial_reg(x, t, M, lamb)` that fits a _regularized_ $M$-th order polynomial to the periodic data, as discussed in the lectures, where `lamb` is the regularization term _lambda_. (Note that 'lambda' cannot be used as a variable name in Python since it has a special meaning). The error function to minimize w.r.t. $\mathbf{w}$:

$E(\mathbf{w}) = \frac{1}{2} (\mathbf{\Phi}\mathbf{w}- \mathbf{t})^T(\mathbf{\Phi}\mathbf{w}- \mathbf{t}) + \frac{\lambda}{2} \mathbf{w}^T \mathbf{w}$


The function should return $\mathbf{w}$ and $ \mathbf{\Phi} $.

In [ ]:
def fit_polynomial_reg(x, t, m, lamb):
  Phi = designmatrix(x, m)
  diag_lambda = np.eye(m + 1) * lamb
  w_ml = np.linalg.inv(diag_lambda + Phi.T @ Phi) @ Phi.T @ t
  
  return w_ml, Phi


In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
m = 2
lamb = 0.1
w, Phi = fit_polynomial_reg(x,t,m, lamb)

assert w.shape == (m+1,), "The shape of w is incorrect"
assert Phi.shape == (N, m+1), "The shape of w is incorrect" 



b) (5 points) What changes do you notice in $\mathbf{w}$ and $\mathbf{\Phi}$ after introducing the regularization term? Why is this happening? 

(Write no more than 5 lines. For example, you can consider the simple test case with $t = 0.3*x^2 + 2.5$)

The overall values of $\mathbf{w}$ get close to 0 when adding the regularization term. This is because we are minimising the norm of the vector $\mathbf{w}$ in the error function.

The matrix $\mathbf{\Phi}$ does not change since it only contains information about the input. (i.e. the values of the basis functions applied to the input $\mathbf{x}$ component-wise) 

### 1.5 Model selection by cross-validation (15 points)

Relevant material for this part:

* Lecture 4.1 and 4.2 (Model Selection, Bias Variance Decomposition)

Use cross-validation to find a good choice of $M$ and $\lambda$, given a dataset of $N=10$ datapoints generated with `gen_sine(10)`. You should write a function that tries (loops over) a reasonable range of choices of $M$ and $\lambda$, and returns the choice with the best cross-validation error. In this case you use $K=5$ folds.

You can let $M \in (0, 1, ..., 10)$, and let $\lambda \in (e^{-10}, e^{-9}, ..., e^{0})$.

a) (5 points) First of all, write a method `pred_error(x_train, x_valid, t_train, t_valid, M, lamb)` that compares the prediction of your method `fit_polynomial_reg` for a given set of parameters $M$ and $\lambda$ to `t_valid`. It should return the prediction error for a single fold.

In [ ]:
def pred_error(x_train, x_valid, t_train, t_valid, M, reg):
  w = fit_polynomial_reg(x_train, t_train, m, reg)[0]
  prediction = polynomial(x_valid, w)
  pred_err = np.sum(prediction - t_valid)
    
  return pred_err


In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
M = 2
reg = 0.1
pred_err = pred_error(x[:-2], x[-2:], t[:-2], t[-2:], M, reg)

assert pred_err < 0.001, "pred_err is too big"



b) (10 points) Now write a method find_best_m_and_lamb(x, t) that finds the best values for $M$ and $\lambda$. The method should return the best $M$ and $\lambda$. To get you started, here is a method you can use to generate indices of cross-validation folds.

In [ ]:
def kfold_indices(N, k):
  all_indices = np.arange(N,dtype=int)
  np.random.shuffle(all_indices)
  idx = [int(i) for i in np.floor(np.linspace(0,N,k+1))]
  train_folds = []
  valid_folds = []
  for fold in range(k):
    valid_indices = all_indices[idx[fold]:idx[fold+1]]
    valid_folds.append(valid_indices)
    train_folds.append(np.setdiff1d(all_indices, valid_indices))
  return train_folds, valid_folds


In [ ]:
def find_best_m_and_lamb(x, t):
  M_set = list(range(1, 11))
  Lambda_set = [np.exp(-i) for i in range(11)]

  current_error = float('inf')
  M_best = -1
  lamb_best = -1

  for m in M_set:
    for lamb in Lambda_set:
      train_folds, valid_folds = kfold_indices(x.shape[0], 5) # use 5 folds
      accumulated_error = 0
      for train_fold, valid_fold in zip(train_folds, valid_folds):
        x_train = x[train_fold]
        t_train = t[train_fold]
        x_valid = x[valid_fold]
        t_valid = t[valid_fold]

        accumulated_error += pred_error(x_train, x_valid, t_train, t_valid, m, lamb) # sum the errors across all folds

      if accumulated_error < current_error:
        current_error = accumulated_error
        M_best = m
        lamb_best = lamb
  
  return M_best, lamb_best

In [ ]:
### This is not an empty cell (You don't need to care about it).

### 1.6 Why grid search? (5 points)

Grid search is an commonly-used technique to tune hyper-parameters in a model.
Considering the case described in the previous step of this assignment, running a grid search over the possible parameter values (10 possible values for both $M$ and $\lambda$), results in two nested loops exploring $10 \times 10 = 100$ different configurations for the model. 

a) (3 points) Why do we want to optimize by changing the two hyperparameters at the same time, and not in a sequential way? We could initialise all parameters randomly, fix one parameter at a time and iterate over the other, resulting in only $10 + 10 = 20$ experiments!

This method doesn't take into account the fact that the hyperparameters influence each others performance because this approach does not take into consideration the correlation factor between the 2 hyperparameters. This method finds the best lambda for some M, and finds the best value of M for this specific lambda, but this does not necessarily have to be the best overall combination.

For more complex models, the number of combinations easily explodes with the number of parameters. For example, with 5 parameters we would run $10 \times 10 \times 10 \times 10 \times 10 = 100,000$ experiments.

b) (2 points) Try to think or find in literature one alternative to grid search to tune hyper-parameters more efficiently. Explain very briefly (2-3 lines max) how this method avoids the combinatorial explosion we have see in grid search.

Genetic algorithms are often used for hyperparameter tuning. This approach works by randomly generating a set of hyperparameters and selecting, recombining and mutating sets of hyperparameter based on performance. This way combinatorial explosion is avoided because we don't explore the whole search space, since the algorithm directs us to good combinations of hyperparameters. 
(Liashchynskyi, Petro & Liashchynskyi, Pavlo. (2019). Grid Search, Random Search, Genetic Algorithm: A Big Comparison for NAS.)

### 1.7 Plot best cross-validated fit (5 points)

For some dataset with $N = 10$, plot the model with the optimal $M$ and $\lambda$ according to the cross-validation error, using the method you just wrote. In addition, the plot should show the dataset itself and the function that we try to approximate. Let the plot make clear which $M$ and $\lambda$ were found.

In [ ]:
# define datapoints
data_x, data_t = gen_sine(10)
x_ticks = np.arange(0,2*np.pi,0.1)   
sin_y = np.sin(x_ticks)

plotcount = 221

# generate plots of datapoints and sine
plt.subplot(plotcount)
plotcount = plotcount + 1
# plot data and sine
plt.plot(data_x, data_t, 'ro')
plt.plot(x_ticks, sin_y)

best_m, best_lambda = find_best_m_and_lamb(data_x, data_t)
# calculate weights and plot polynomial
w = fit_polynomial_reg(data_x, data_t, best_m, best_lambda)[0]
plt.plot(x_ticks,polynomial(x_ticks, w))
# set labels 
plt.xlabel('x')
plt.ylabel('t')
plt.text(4, 0.5, 'M=%s'%(order), fontsize = 10)

plt.show()

## Part 2: Bayesian Linear (Polynomial) Regression

Relevant material for this part:

* Lecture 4.4 and 4.5 (Sequential Bayesian Learning, Bayesian Predictive Distributions)
* Section 1.1 and 3.3 from Bishop's book Pattern recognition for machine learning.
* Mathematics for machine learning (Section 8.4)

### 2.1 Sine 2 (5 points)

Write a function `gen_sine2(N)` that behaves identically to `gen_sine(N)` except that the generated values $x_i$ are not linearly spaced, but drawn from a uniform distribution between $0$ and $2 \pi$.

In [ ]:
def gen_sine2(n):
  # define x as vector of n values, randomly distributed between 0 and 2pi 
  x = np.random.uniform(0,2*np.pi, n)
  x = np.sort(x)
  # define y as sine of x plus random noise
  t = np.sin(x) + np.random.normal(0, 0.25, n)

  return x, t

In [ ]:
### Test your function
np.random.seed(42)
N = 10
x, t = gen_sine2(N)

assert x.shape == (N,), "the shape of x is incorrect"
assert t.shape == (N,), "the shape of t is incorrect"



### 2.2 Compute Posterior (15 points)

You're going to implement a Bayesian linear regression model, and fit it to the periodic data. Your regression model has a zero-mean isotropic Gaussian prior over the parameters, governed by a single (scalar) precision parameter $\alpha$, i.e.:

$$p(\mathbf{w} \;|\; \alpha) = \mathcal{N}(\mathbf{w} \;|\; 0, \alpha^{-1} \mathbf{I})$$

The covariance and mean of the posterior are given by:

$$\mathbf{S}_N= \left( \alpha \mathbf{I} + \beta \mathbf{\Phi}^T \mathbf{\Phi} \right)^{-1} $$
$$\mathbf{m}_N = \beta\; \mathbf{S}_N \mathbf{\Phi}^T \mathbf{t}$$

where $\alpha$ is the precision of the predictive distribution, and $\beta$ is the noise precision. 
See MLPR chapter 3.3 for background.

Write a method `fit_polynomial_bayes(x, t, M, alpha, beta)` that returns the mean $\mathbf{m}_N$ and covariance $\mathbf{S}_N$ of the posterior for a $M$-th order polynomial. In addition it should return the design matrix $\mathbf{\Phi}$. The arguments `x`, `t` and `M` have the same meaning as in question 1.2.

In [ ]:
def fit_polynomial_bayes(x, t, M, alpha, beta):
  # define matrices 
  Phi = designmatrix(x,M)
  Phi_T = Phi.transpose()
  I = np.identity(M+1)
  # calculate S_n and m_n 
  S = np.linalg.inv((alpha*I + beta*np.matmul(Phi_T, Phi)))
  m = beta*np.matmul(S, np.matmul(Phi_T,t))
  
  return m, S, Phi

In [ ]:
### Test your function
N = 10
x = np.linspace(-1, 1, N)
t = 0.3*np.square(x) + 2.5
M = 2
alpha = 0.6
beta = 16
m, S, Phi = fit_polynomial_bayes(x, t, M, alpha, beta)

assert m.shape == (M+1,), "the shape of m is incorrect" 
assert S.shape == (M+1, M+1), "the shape of S is incorrect"
assert Phi.shape == (N, M+1), "the shape of Phi is incorrect"



### 2.3 Prediction (10 points)

The predictive distribution of Bayesian linear regression is:

$$ p(t \;|\; \mathbf{x}, \mathbf{t}, \alpha, \beta) = \mathcal{N}(t \;|\; \mathbf{m}_N^T \phi(\mathbf{x}), \sigma_N^2(\mathbf{x}))$$

$$ \sigma_N^2 = \frac{1}{\beta} + \phi(\mathbf{x})^T \mathbf{S}_N \phi(\mathbf{x}) $$

where $\phi(\mathbf{x})$ are the computed features for a new datapoint $\mathbf{x}$, and $t$ is the predicted variable for datapoint $\mathbf{x}$. 

Write a function that `predict_polynomial_bayes(x, m, S, beta)` that returns the predictive mean, variance and design matrix $\mathbf{\Phi}$ given a new datapoint `x`, posterior mean `m`, posterior variance `S` and a choice of model variance `beta`.

In [ ]:
def predict_polynomial_bayes(x, m, S, beta):
  mean = np.empty(shape=x.shape)
  sigma = np.empty(shape=x.shape)
  Phi = np.empty(shape=(x.shape[0], m.shape[0]))

  for i in range(x.shape[0]):
    Phi[i] = np.array([x[i] ** power for power in range(0, m.shape[0])])
    mean[i] = m.T @ Phi[i]
    sigma[i] = 1 / beta + Phi[i].T @ S @ Phi[i]

  return mean, sigma, Phi


In [ ]:
### Test your function
np.random.seed(42)
N = 10
x = np.linspace(-1, 1, N)
m = np.random.rand(3)
S = np.random.rand(3, 3)
beta = 16
mean, sigma, Phi = predict_polynomial_bayes(x, m, S, beta)

assert mean.shape == (N,), "the shape of mean is incorrect"
assert sigma.shape == (N,), "the shape of sigma is incorrect"
assert Phi.shape == (N, m.shape[0]), "the shape of Phi is incorrect"



### 2.4 Plot predictive distribution (10 points)

a) (5 points) Generate 10 datapoints with `gen_sine2(10)`. Compute the posterior mean and covariance for a Bayesian polynomial regression model with $M=4$, $\alpha=\frac{2}{5}$ and $\beta=\frac{1}{0.25^2}$.
Plot the Bayesian predictive distribution, where you plot (for $x$ between 0 and $2 \pi$) $t$'s predictive mean and the predictive standard deviation using `plt.fill_between(..., alpha=0.1)` (the alpha argument induces transparency).

Include the datapoints in your plot.


In [ ]:
M = 4
alpha = 2 / 5
beta = 16

# define datapoints
x, t = gen_sine2(10)

plotcount = 221

# generate plots of datapoints and sine
plt.subplot(plotcount)
plotcount = plotcount + 1
# plot data and sine
plt.plot(x, t, 'ro')

posterior_mean, covariance, _ = fit_polynomial_bayes(x, t, M, alpha, beta)
predictive_mean, predictive_variance, _ = predict_polynomial_bayes(x, posterior_mean, covariance, beta)

plt.plot(x, predictive_mean)
plt.fill_between(x=x, y1=predictive_mean, y2=predictive_mean + predictive_variance)#, alpha=0.1)
plt.fill_between(x=x, y1=predictive_mean, y2=predictive_mean - predictive_variance)#, alpha=0.1)

# plt.plot(x_ticks,polynomial(x_ticks, w))
# # set labels 
plt.xlabel('x')
plt.ylabel('t')
plt.text(4, 0.5, 'M=%s'%(order), fontsize = 10)

plt.show()

b) (5 points) For a second plot, draw 100 samples from the parameters' posterior distribution. Each of these samples is a certain choice of parameters for 4-th order polynomial regression. 
Display each of these 100 polynomials.


In [ ]:
M = 4

mean = np.zeros(M + 1)
covariance = (1 / alpha) * np.eye(M + 1)

N = 100
for i in range(N):
  w = np.random.multivariate_normal(mean, covariance)
  x = np.linspace(-1, 1, N)
  y = polynomial(x, w)
  
  plt.plot(x, y)

plt.show()

### 2.5 Additional questions (10 points)

a) (5 points) Why is $\beta=16$ the best choice of $\beta$ in section 2.4?



For large values of $\beta$ we get a really small variance in the final predictive distribution. Also, for small values of $\beta$ we get a really large variance. This means that we should find a sweet spot in order to avoid underfitting and overfitting.
 

b) (5 points) What problems do we face when it comes to choosing basis functions in linear models?

Choosing a good basis function is really dependent on the task at hand. What are good basis functions really depends on the how the perfect model would look, which is something that you don't know beforehand. Depending on the data distribution we are trying to fit, we might see different results for different basis functions. So, one issue with choosing the best basis function for a particular task is that it gets time consuming because of all the experimantation.
